In [1]:
import math, random
import torch
import librosa
from IPython.display import Audio

In [2]:
class AudioUtil():
    
    @staticmethod
    def open(audio_file):
        sig, sr = librosa.load(audio_file)
        
        return (sig, sr)
    
    @staticmethod
    def rechannel(aud, new_channel):
        
        sig, sr = aud
        
        if (sig.shape[0] == new_channel):
            return aud
        
        if (new_channel == 1):
            resig = sig[:1, :]
        else:
            resig = torch.cat([sig, sig])
            
        return ((resig, sr))
    
    @staticmethod
    def resample(aud, newsr):
        sig, sr = aud
        
        if (sr == newsr):
            return aud
        
        num_channels = sig.shape[0]
        resig = librosa.resample(sr, newsr)(sig[:1, :])
        
        if (num_channels > 1):
            retwo = librosa.resample(sr, newsr)(sig[1:, :])
            resig = torch.cat([resig, retwo])
            
        return ((resig, newsr))
    
    @staticmethod
    def pad_trunc(aud, max_ms):
        sig, sr= aud
        num_rows, sig_len = sig.shape
        max_len = sr // 1000 * max_ms
        
        if (sig_len > max_len):
            sig = sig[:, :max_len]
            
        elif (sig_len < max_len):
            pad_begin_len = random.randint(0, max_len - sig_len)
            pad_end_len = max_len - sig_len - pad_begin_len
            
            pad_begin = torch.zeros((num_rows, pad_begin_len))
            pad_end = torch.zeros((num_rows, pad_end_len))
            
            sig = torch.cat((pad_begin, sig, pad_end), 1)
            
        return (sig, sr)
    
    @staticmethod
    def time_shift(aud, shift_limit):
        sig, sr = aud
        _, sig_len = sig.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        return (sig.roll(shift_amt), sr)
    
    @staticmethod
    def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
        sig, sr = aud
        top_db = 80
        
        spec = librosa.feature.melspectrogram(sr, n_fft=n_fft, hop_length=hop_len,
                                              n_mels=n_mels)(sig)
        
        spec = librosa.amplitude_to_db(top_db=top_db)(spec)
        
        return (spec)

In [3]:
from torch.utils.data import DataLoader, Dataset, random_split

In [ ]:
class SoundDS(Dataset):
    
    def __init__(self, df, data_path):
        self.df = df
        self.data_path = str(data_path)
        self.duration = 4000
        self.sr = 44100
        self.channel = 2
        self.shift_pct = 0.4
        
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        audio_file = self.data_path + self.df.loc[idx, 'relative_path']
        class_id = self.df.loc[idx, 'classID']
        
        aud = AudioUtil.open(audio_file)
        reaud = AudioUtil.resample(aud, self.sr)
        rechan = AudioUtil.rechannel(reaud, self.channel)
        
        dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
        shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
        sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
        
        return sgram, class_id

In [22]:
from IPython.display import clear_output

In [23]:
data = save_mfcc(dataset_path, json_path)
clear_output()

In [24]:
def load_data(dataset: dict):
        
    inputs = np.array(dataset['mfcc'])
    targets = np.array(dataset['labels'])
    
    return inputs, targets

In [25]:
inputs, targets = load_data(data)

C:\Users\GODOVO~1\AppData\Local\Temp/ipykernel_10852/1172913467.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  inputs = np.array(dataset['mfcc'])


ValueError: could not broadcast input array from shape (10,137) into shape (10,)

In [14]:
inputs = inputs.reshape(-1)

In [17]:
inputs.shape

(360,)

In [18]:
from sklearn.model_selection import train_test_split

input_train, input_test, target_train, target_test = train_test_split(inputs, targets, test_size=0.3)
print(input_train.shape, target_train.shape)

(252,) (252,)


In [19]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *

In [20]:
model = Sequential()

model.add(Flatten(input_shape=(inputs.shape[0], inputs.shape[1])))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(12, activation='relu'))
model.summary()

IndexError: tuple index out of range

In [25]:
from tensorflow.keras import optimizers
adam = optimizers.Adam(learning_rate=1e-4)

In [26]:
model.compile(optimizer=adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

hist = model.fit(input_train, target_train,
                 validation_data=(input_test, target_test),
                 epochs=50,
                 batch_size=32)

clear_output()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).